# Get access to Statsbomb data
* See link for [details about Statsbomb datasets](https://github.com/statsbomb/statsbombpy) and how to install `statsbombpy` 
* Run this notebook to create the raw datatset of all the passes  between players.

In [1]:
import pandas as pd
from statsbombpy import sb
from statsbombpy import api_client, public
from statsbombpy.config import DEFAULT_CREDS, PARALLELL_CALLS_NUM
from statsbombpy.helpers import filter_and_group_events, is_relevant, reduce_events

# Get Matches

In [2]:
import statsbombpy
def lineup(match_id):
    dfs_lineup = []
    lineups = statsbombpy.public.lineups(match_id=match_id)
    for l in lineups.values():
        lineup = pd.DataFrame(l["lineup"])
        lineup["country"] = lineup.country.apply(lambda c: c['name'] if str(c) != 'nan' else '')
        lineup['team_name'] = l["team_name"]
        lineup['team_id'] = l["team_id"]
        dfs_lineup.append(lineup)
    return pd.concat(dfs_lineup)

In [3]:
df_competitions = sb.competitions()
pairs_competitions = df_competitions[['competition_id', 'season_id']].values

credentials were not supplied. open data access only


In [4]:
dfs_matches = []
for competition_id, season_id in pairs_competitions:
    try:
        df_match = sb.matches(competition_id=competition_id, season_id=season_id)
        df_match['competition_id'] = competition_id
        df_match['season_id'] = season_id
        dfs_matches.append(df_match)
    except Exception as e:
        print(f"{competition_id},{season_id} - Exception: {e}")
df_matches = pd.concat(dfs_matches)
matches_ids = df_matches['match_id'].values

credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
16,76 - Exception: 'DataFrame' object has no attribute 'competition'
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied.

# Get Events (in match)

Change `is_sample_matches` to `False` to get all passes (around 3M passes)

In [5]:
dfs_pass_events = []
is_sample_matches = True
if is_sample_matches:
    sample_matches = matches_ids[:5]
else:
    sample_matches = matches_ids
    
for match_id in sample_matches:
    try:
        dfs_pass_event = sb.events(match_id=match_id)        
        dfs_pass_event['match_id'] = match_id
        dfs_pass_event['pass_recipient'] = dfs_pass_event['pass'].dropna().apply(pd.Series)['recipient'].dropna().apply(pd.Series)['name']
        dfs_pass_event['pass_recipient_id'] = dfs_pass_event['pass'].dropna().apply(pd.Series)['recipient'].dropna().apply(pd.Series)['id']  
        dfs_pass_event['pass_length'] = dfs_pass_event['pass'].dropna().apply(pd.Series)['length']
        dfs_pass_event['pass_type'] = dfs_pass_event['pass'].dropna().apply(pd.Series)['type'].dropna().apply(pd.Series)['name']        
        # extract Player ID
        l = {} 
        x = dfs_pass_event[dfs_pass_event.type=='Substitution']['substitution'].apply(pd.Series)['replacement'].values
        l.update({i['name']: i['id'] for i in x})
        for x in dfs_pass_event[dfs_pass_event.type=='Starting XI']['tactics']:
            l.update({i['player']['name']: i['player']['id'] for i in x['lineup']})
        df_players = pd.DataFrame.from_dict(l, orient='index',columns=['playerID'])
        dfs_pass_event = pd.merge(dfs_pass_event, df_players, left_on='player', right_index=True)
        
        dfs_pass_events.append(dfs_pass_event)
    except Exception as e:
        print(f"{match_id} - Exception: {e}")
dfs_pass_events = pd.concat(dfs_pass_events)  
display(f"dfs_pass_events: {dfs_pass_events.shape}")  # (3025822, 46)

credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only


'dfs_pass_events: (18116, 44)'

In [6]:
_event_cols = ([ 'pass_length','pass_type'])
cols = ['match_id','timestamp','possession_team','player','playerID', 'type',  'pass_recipient', 'pass_recipient_id',]  #'id','minute','related_events', 'substitution',  'foul_won', 'dribble', 'shot', 'goalkeeper', 'clearance', 'duel',
event_cols = cols + _event_cols

dfs_pass_event[event_cols].sample(5)

,match_id,timestamp,possession_team,player,playerID,type,pass_recipient,pass_recipient_id,pass_length,pass_type
1017,18242,00:03:07.915,Barcelona,Sergio Busquets i Burgos,5203,Ball Receipt*,NaN,NaN,NaN,NaN
2256,18242,00:38:24.030,Juventus,Paul Pogba,20004,Carry,NaN,NaN,NaN,NaN
3423,18242,00:34:40.061,Juventus,Luis Alberto Suárez Díaz,5246,Injury Stoppage,NaN,NaN,NaN,NaN
2171,18242,00:31:27.787,Barcelona,Sergio Busquets i Burgos,5203,Carry,NaN,NaN,NaN,NaN
2509,18242,00:20:15.120,Barcelona,Jordi Alba Ramos,5211,Carry,NaN,NaN,NaN,NaN


# Export Passes Data

In [ ]:
# dfs_pass_events.to_csv('../data/raw/statsbomb/player_pass.csv', sep='|')
dfs_pass_events[dfs_pass_events['type']=='Pass'][event_cols].to_parquet(
    path='../data/raw/statsbomb/player_pass.parq',
    compression=None,
    engine='fastparquet'
)
